In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Time-Series-Analysis-for-Whether-Dataset'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
data = pd.read_csv('./Time-Series Analysis Dataset.csv')

In [3]:
data

       datetime_local  temperature  humidity  dew_point  wind_bearing  \
0    01-10-2019 00:00         26.0       1.0      22.01         101.0   
1    01-10-2019 01:00         25.0       1.0      22.06         105.0   
2    01-10-2019 02:00         25.0       1.0      22.06         106.0   
3    01-10-2019 03:00         24.0       1.0      21.87         106.0   
4    01-10-2019 04:00         23.0       1.0      21.53         100.0   
..                ...          ...       ...        ...           ...   
643               NaN          NaN       NaN        NaN           NaN   
644               NaN          NaN       NaN        NaN           NaN   
645               NaN          NaN       NaN        NaN           NaN   
646               NaN          NaN       NaN        NaN           NaN   
647               NaN          NaN       NaN        NaN           NaN   

     wind_speed  wind_gust  pressure  uv_index  ozone  precip_intensity  \
0          3.08       6.42   1008.45       0.0  

In [4]:
data = data['temperature'].dropna()

In [5]:
data = torch.from_numpy(np.array(data['temperature'].dropna().tolist())).view(1,-1)

In [6]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from tqdm import tqdm
import cv2
from torch.optim import *
# Preproccessing
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    Normalizer,
    Binarizer
)
# Decomposition
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
# Feature Selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel
# Model Eval
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error
# Other
import pickle
import wandb

PROJECT_NAME = 'Time-Series-Analysis-for-Whether-Dataset'
device = 'cuda:0'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [7]:
data = pd.read_csv('./Time-Series Analysis Dataset.csv')

In [8]:
data = torch.from_numpy(np.array(data['temperature'].dropna().tolist())).view(1,-1)

In [9]:
data

tensor([[26., 25., 25., 24., 23., 23., 24., 25., 25., 27., 28., 30., 31., 32.,
         33., 33., 33., 33., 32., 30., 28., 27., 27., 26., 26., 26., 25., 25.,
         24., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 33., 33.,
         32., 31., 29., 28., 28., 27., 26., 26., 26., 25., 24., 24., 25., 25.,
         26., 28., 30., 31., 31., 33., 34., 35., 34., 33., 31., 30., 29., 28.,
         28., 27., 26., 26., 25., 24., 24., 24., 24., 25., 25., 27., 28., 30.,
         31., 32., 33., 33., 34., 33., 31., 28., 28., 27., 27., 26., 25., 25.,
         24., 23., 22., 22., 23., 24., 24., 26., 28., 30., 31., 32., 33., 34.,
         34., 33., 32., 30., 28., 27., 26., 26., 25., 24., 23., 23., 22., 22.,
         23., 24., 25., 27., 29., 31., 32., 33., 33., 33., 34., 33., 32., 30.,
         28., 27., 26., 26., 25., 24., 24., 23., 23., 22., 23., 24., 24., 25.,
         27., 29., 30., 32., 32., 32., 32., 31., 30., 30., 28., 27., 26., 26.,
         25., 24., 24., 23., 21., 21., 22., 22., 23.

In [10]:
data_input = data[:1,:-1].to(device).float()
data_target = data[:1,1:].to(device).float()

In [11]:
data_input

tensor([[26., 25., 25., 24., 23., 23., 24., 25., 25., 27., 28., 30., 31., 32.,
         33., 33., 33., 33., 32., 30., 28., 27., 27., 26., 26., 26., 25., 25.,
         24., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 33., 33.,
         32., 31., 29., 28., 28., 27., 26., 26., 26., 25., 24., 24., 25., 25.,
         26., 28., 30., 31., 31., 33., 34., 35., 34., 33., 31., 30., 29., 28.,
         28., 27., 26., 26., 25., 24., 24., 24., 24., 25., 25., 27., 28., 30.,
         31., 32., 33., 33., 34., 33., 31., 28., 28., 27., 27., 26., 25., 25.,
         24., 23., 22., 22., 23., 24., 24., 26., 28., 30., 31., 32., 33., 34.,
         34., 33., 32., 30., 28., 27., 26., 26., 25., 24., 23., 23., 22., 22.,
         23., 24., 25., 27., 29., 31., 32., 33., 33., 33., 34., 33., 32., 30.,
         28., 27., 26., 26., 25., 24., 24., 23., 23., 22., 23., 24., 24., 25.,
         27., 29., 30., 32., 32., 32., 32., 31., 30., 30., 28., 27., 26., 26.,
         25., 24., 24., 23., 21., 21., 22., 22., 23.

In [12]:
data_target

tensor([[25., 25., 24., 23., 23., 24., 25., 25., 27., 28., 30., 31., 32., 33.,
         33., 33., 33., 32., 30., 28., 27., 27., 26., 26., 26., 25., 25., 24.,
         23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 33., 33., 32.,
         31., 29., 28., 28., 27., 26., 26., 26., 25., 24., 24., 25., 25., 26.,
         28., 30., 31., 31., 33., 34., 35., 34., 33., 31., 30., 29., 28., 28.,
         27., 26., 26., 25., 24., 24., 24., 24., 25., 25., 27., 28., 30., 31.,
         32., 33., 33., 34., 33., 31., 28., 28., 27., 27., 26., 25., 25., 24.,
         23., 22., 22., 23., 24., 24., 26., 28., 30., 31., 32., 33., 34., 34.,
         33., 32., 30., 28., 27., 26., 26., 25., 24., 23., 23., 22., 22., 23.,
         24., 25., 27., 29., 31., 32., 33., 33., 33., 34., 33., 32., 30., 28.,
         27., 26., 26., 25., 24., 24., 23., 23., 22., 23., 24., 24., 25., 27.,
         29., 30., 32., 32., 32., 32., 31., 30., 30., 28., 27., 26., 26., 25.,
         24., 24., 23., 21., 21., 22., 22., 23., 25.

In [13]:
class Model(Module):
    def __init__(self,hidden=512):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,hidden)
        self.lstm2 = LSTMCell(hidden,hidden)
        self.linear = Linear(hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)

In [14]:
model = Model().to(device)

In [15]:
criterion = MSELoss()

In [16]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [17]:
epochs = 100

In [18]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].detach().numpy()
        n = data_target.shape[1]
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_target.view(-1).detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('img.png'))})

In [19]:
class Model(Module):
    def __init__(self,hidden=512):
        super().__init__()
        self.hidden = hidden
        self.lstm1 = LSTMCell(1,hidden)
        self.lstm2 = LSTMCell(hidden,hidden)
        self.linear = Linear(hidden,1)
    
    def forward(self,X,future=0):
        preds = []
        batch_size = X.shape[0]
        h_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t1 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        h_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        c_t2 = torch.zeros(batch_size,self.hidden,dtype=torch.float32).to(device)
        for X_batch in X.split(1,dim=1):
            X_batch = X_batch.to(device)
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        for i in range(future):
            h_t1,c_t1 = self.lstm1(X_batch,(h_t1,c_t1))
            h_t1 = h_t1.to(device)
            c_t1 = c_t1.to(device)
            h_t2,c_t2 = self.lstm2(h_t1,(h_t2,c_t2))
            h_t2 = h_t2.to(device)
            c_t2 = c_t2.to(device)
            pred = self.linear(h_t2)
            preds.append(pred)
        preds = torch.cat(preds,dim=1)
        return preds

In [20]:
model = Model().to(device)

In [21]:
criterion = MSELoss()

In [22]:
optimizer = LBFGS(model.parameters(),lr=0.001)

In [23]:
epochs = 100

In [24]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    def closure():
        optimizer.zero_grad()
        preds = model(data_input)
        loss = criterion(preds,data_target)
        wandb.log({'Loss':loss.item()})
        loss.backward()
        return loss
    optimizer.step(closure)
    with torch.no_grad():
        future = 100
        preds = model(data_input,future)
        loss = criterion(preds[:,:-future],data_target)
        wandb.log({'Val Loss':loss.item()})
        preds = preds[0].detach().numpy()
        n = data_target.shape[1]
        plt.figure(figsize=(12,6))
        plt.plot(np.arange(n),data_target.view(-1).detach().numpy())
        plt.plot(np.arange(n,n+future),preds[n:])
        plt.savefig('img.png')
        plt.close()
        wandb.log({'Img':wandb.Image(cv2.imread('img.png'))})